In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
### Output = (N - F + 2P)/S + 1

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, len(dataset.classes))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def get_accuracy(model, dataloader):
  "Gets accuracy for final accuracy and accuracy plots"
  correct=0
  total=0
  with torch.no_grad():
    for imgs, labels in dataloader:

      #To Enable GPU Usage
      if use_cuda and torch.cuda.is_available():
          # print("consist")
          imgs = imgs.cuda()
          labels = labels.cuda()

      outputs = model(imgs)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      # print(predicted)#weird size mismatch
      # print(labels)

      correct += (predicted == labels).sum().item()#adds sum of them
  return correct/total

In [ ]:
import time

def train_plot(model, num_epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  iters,losses,train_acc,valid_acc =[],[],[],[]

  start_time=time.time()
  for epoch in range(num_epochs):  # Loop over the dataset multiple times
    running_loss = 0.0
    model.train()  # Set the model to training mode

    for i, data in enumerate(train_loader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data


      #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          # model.cuda()
          # print("consist")
          inputs = inputs.cuda()
          labels = labels.cuda()
        #############################################

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            iters.append(epoch * 100 + i)
            losses.append(running_loss / 100)
            train_acc.append(get_accuracy(model, train_loader))
            valid_acc.append(get_accuracy(model, val_loader))

            print(f'Epoch {epoch + 1}, Minibatch {i + 1}, Loss: {running_loss / 100:.3f}')
            print("\tTime Elapsed: % 6.2f s " % (time.time()-start_time))
            running_loss = 0.0

#done training
  print(iters)
  print(losses)

  plt.title("Training Curve")
  plt.plot(iters, losses, label="Train")
  plt.xlabel("Iterations")
  plt.ylabel("Loss")
  plt.show()


  plt.title("Accuracy Train Curve")
  plt.plot(iters, valid_acc, label="Validation")
  plt.plot(iters, train_acc, label="Training")
  plt.legend(loc='best')
  plt.xlabel("Iterations")
  plt.ylabel("Accuracy")
  plt.show()
  print("Final Training Accuracy: {}".format(train_acc[-1]))
  print("Final Validation Accuracy: {}".format(valid_acc[-1]))



In [ ]:
model = SimpleCNN()

use_cuda = True


if use_cuda and torch.cuda.is_available():
  model.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

train_plot(model, num_epochs=1)

In [ ]:
test_acc=get_accuracy(model, test_loader)
print(f'Final Test Accuracy: {100 * test_acc:.2f}%')